# Testing OpenCV algorithms

In [1]:
import numpy as np
import cv2
import os

In [3]:
# Load the pathimage
path = 'Data/retinal-lesions-v20191227/images_896x896'

for idx, imp in enumerate(os.listdir(path)):
    im_path = os.path.join(path, imp)
    orig_img = cv2.imread(im_path)
    img = orig_img.copy()

    # Extract the green plane
    green_plane = img[:, :, 1]

    # Define the FA-CLAHE algorithm
    faclahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))

    # Apply the FA-CLAHE algorithm to the green plane
    faclahe_img = faclahe.apply(green_plane)

    # Replace the green plane with the enhanced green plane
    img[:, :, 1] = faclahe_img

    # Display the original and enhanced images
    cv2.imshow('Original Image', orig_img)
    cv2.waitKey(0)
    cv2.imshow('FA-CLAHE Enhanced Image', faclahe_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [9]:
im_path = 'Data/retinal-lesions-v20191227/lesion_segs_896x896/original/9_left/cotton_wool_spots.png'
# Load the input image in grayscale mode
img = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE)

# Apply binary thresholding to convert the image to a binary image
ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Get the connected components of the binary image
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

# Iterate over the connected components and display them one by one
for i in range(1, num_labels):
    # Create a binary image for the current component
    component = (labels == i).astype(np.uint8) * 255
    
    # Display the current component
    cv2.imshow('Component {}'.format(i), component)
    
    # Wait for a key event
    cv2.waitKey(0)

    # Close all open windows
    cv2.destroyAllWindows()


In [2]:
os.path.exists('mask.png')

True